<a href="https://colab.research.google.com/github/batiukmaks/NLP-Labs/blob/main/labs/LPNLP06_Spellchecker_quest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spellchecker quest

Хтось наробив помилок у віршах Тараса Шевченка. Наша задача -- виправити ці помилки і прочитати приховане повідомлення.

## Задача

Ви отримаєте тренувальні та тестувальні дані.

Тренувальні дані знаходяться в полі `lab.train_text`. Це звичайний нерозмічений текст. На ньому необхідно натренувати мовну модель. Підійде будь-яка. Я би радив feed-forward нейрону модель з токенізацією по літерах, бо це те, що ми проходили на останній лекції. Але n-грамна теж має спрацювати.

Тестувальні дані знаходиться в полі `lab.test_items`. Приклад одного елемента:

```json
{
  "text": "Співали б прозу, та по ножах,",
  "error_start": 23,
  "error_end": 28,
  "error": "ножах",
  "corrections": [
    "ногах",
    "йотах",
    "єнотах",
    "ножах",
    "нотах"
  ]
}
```

`error_start` та `error_end` вказують на місцезнаходження помилки в тексті (в символах). У данному прикладі, помилкою є `text[23:28]`, тобто слово "ножах".

`corrections` -- це список можливих виправлень.

Ваша задача -- обрати правильне виправлення серед запропонованих.


## Приховане повідомлення

Один приклад в `lab.test_items` дає можливість прочитати одну літеру прихованого повідомлення. Для цього знайдіть різницю між літерами слова з помилкою (`error`) та обраним виправленням. Надрукуйте цю літеру. Якщо слово з помилкою направді правильне, а таке теж буває, надрукуйте пробіл. Приклади:

```
Error               Correction     To print
-------------------------------------------
привіт               приліт        л
пні                  поні          о
баллет               балет         л
привіт               привіт        (space)
```

Приховане повідомлення, яке ви побачите в результаті це рядок з віршу одного з українських авторів.

Відповідь на квест -- ім'я автора/ки у форматі "Ім'я Прізвище".

Полетіли! 🚀

In [1]:
!pip install --quiet --ignore-installed http://nlp.band/static/pypy/lpnlp-2023.10.2-py3-none-any.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 6.0 MB/s eta 0:00:00


In [2]:
import lpnlp

lab = lpnlp.start(
    email="maksym.batiuk.kn.2021@lpnu.ua",                   # <----------- Заповніть це поле
    lab="quest_spellchecker"
    )

Удачі!


In [ ]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence

from tqdm import tqdm

## Мовна модель

Натренуйте свою мовну модель тут

In [3]:
len(lab.train_text)

414268

In [4]:
print(lab.train_text[:330])

﻿ПРИЧИННА

Реве та стогне Дніпр широкий,
Сердитий вітер завива,
Додолу верби гне високі,
Горами хвилю підійма.
І блідий місяць на ту пору
Із хмари де-де виглядав,
Неначе човен в синім морі,
То виринав, то потопав.
Ще треті півні не співали,
Ніхто нігде не гомонів,
Сичі в гаю перекликались,
Та ясен раз у раз скрипів.


In [32]:
class Vocabulary:

  def __init__(self, tokens, unk_token="<unk>"):
    self.unk_token = unk_token
    self.unk_index = 0
    self.tokens = [unk_token] + tokens
    self._itos = [unk_token] + list(set(tokens))
    self._stoi = {token: index for index, token in enumerate(self._itos)}

  def stoi(self, token: str) -> int:
    """Return token index or `<unk>` index if `token` is not in the vocab.
    """
    return self._stoi.get(token, self.unk_index)


  def itos(self, index: int) -> str:
    """Return token by its `index`.

    Raise LookupError if `index` is out of vocabulary range.
    """

    return self._itos[index]


  def __len__(self) -> int:
    return len(self._itos)

In [19]:
def get_sequences(tokens, context_len):
    for i in range(context_len, len(tokens)):
        context = tokens[i - context_len : i]
        target = tokens[i]

        yield context, target

def tokenize(text):
    return list(text.lower())

def vectorize(tokens, vocab: Vocabulary):
    return torch.tensor([vocab.stoi(token) for token in tokens], dtype=torch.long)

In [20]:
class Model(nn.Module):
    def __init__(
            self,
            vocab_size,
            embed_dim,
            context_len,
            hidden_dim,
    ):
        super().__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.context_len = context_len
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.W = nn.Linear(context_len * embed_dim, hidden_dim)
        self.U = nn.Linear(hidden_dim, vocab_size)

    def forward(self, X_indices):
        X = self.embed(X_indices)
        e = X.view(-1, self.context_len * self.embed_dim)

        h = torch.tanh(self.W(e)) # TODO: Consider other activation functions
        logits = self.U(h)

        log_probs = torch.log_softmax(logits, dim=-1)

        return log_probs

In [22]:
def batch_it(sequences, batch_size, vocab):
    batch_X = []
    batch_y = []
    for context, target in sequences:
        batch_X.append(vectorize(context, vocab))
        batch_y.append(vocab.stoi(target))

        if len(batch_X) == batch_size:
            # Pad the sequences in batch_X
            X_padded = pad_sequence(batch_X, batch_first=True)
            yield X_padded, torch.tensor(batch_y, dtype=torch.long)
            batch_X = []
            batch_y = []

    if batch_X:
        X_padded = pad_sequence(batch_X, batch_first=True)
        yield X_padded, torch.tensor(batch_y, dtype=torch.long)


def train(
        vocab,
        vocab_size,
        embed_dim,
        context_len,
        hidden_dim,
        learning_rate,
        num_epochs,
        batch_size,
):
    model = Model(
        vocab_size=vocab_size,
        embed_dim=embed_dim,
        context_len=context_len,
        hidden_dim=hidden_dim,
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.NLLLoss()

    sequences = list(get_sequences(vocab.tokens, context_len))
    batches = list(batch_it(sequences, batch_size, vocab))

    for epoch in range(num_epochs):
        total_loss = 0.0

        for X_batch, y_batch in tqdm(batches, leave=False):
            optimizer.zero_grad()
            log_probs = model(X_batch)
            loss = loss_fn(log_probs.view(-1, model.vocab_size), y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"[Epoch {epoch + 1}/{num_epochs}] | Loss: {total_loss / len(sequences)}")

    return model

In [44]:
vocab = Vocabulary(tokenize(lab.train_text))

model = train(
    vocab=vocab,
    vocab_size=len(vocab),
    embed_dim=64,
    context_len=10,
    hidden_dim=128,
    learning_rate=0.001,
    num_epochs=20,
    batch_size=256
)

[Epoch 1/20] | Loss: 0.008942597024767822


[Epoch 2/20] | Loss: 0.007968766632973393


[Epoch 3/20] | Loss: 0.007645278998222103


[Epoch 4/20] | Loss: 0.0074587149498837975


[Epoch 5/20] | Loss: 0.00733488782817987


...

[Epoch 16/20] | Loss: 0.006866633839254427


[Epoch 17/20] | Loss: 0.006848597898688142


[Epoch 18/20] | Loss: 0.006831793031967862


[Epoch 19/20] | Loss: 0.006815969714795329


[Epoch 20/20] | Loss: 0.006801056815374598


# Читаємо між рядків

In [45]:
import math
import collections
from typing import List, Tuple


# Допоміжна фунція:
def get_letter(w1: str, w2: str) -> str:
    """Повертає літеру, якої відрізняються слова або пробіл для однакових слів.
    """

    letters1 = collections.Counter(w1)
    letters2 = collections.Counter(w2)

    diff = letters1 - letters2
    if len(diff) != 1:
        return " "

    return diff.most_common()[0][0]


def score_text(text: str, model, vocab) -> float:
    """Повертає perplexity або log-probability для тексту. """

    tokens = tokenize(text)
    total_log_prob = 0.0

    for context, target in get_sequences(tokens, model.context_len):
        X = vectorize(context, vocab)
        target = vectorize([target], vocab)[0]
        log_probs = model(X)
        target_log_prob = log_probs[0, target]
        total_log_prob += target_log_prob

    return torch.exp(-torch.tensor(total_log_prob) / len(tokens)).item()

# Можете змінювати параметри та весь цей код, якщо потрібно
def solve(model, vocab, test_items) -> Tuple[List[str], str]:
    """Повертає список виправлених слів для кожного з текстів в test_items та
    секретне повідомлення.
    """

    choices = []
    secret = []

    for item in test_items:
        scores = []
        for corr in item['corrections']:

            # Підставляємо слово-кандидат в текст
            text = item['text'][:item['error_start']] + corr + item['text'][item['error_end']:]

            # Рахуємо score тексту
            score = score_text(text, model, vocab)
            scores.append(score)

            # print(f'{score:.4f} {text}')

        # Сортуємо кандидатів на виправлення за score
        result = sorted(zip(scores, item['corrections']), key=lambda x: x[0])

        # Обираємо найкращу заміну
        best = result[0]
        best_word = best[1]
        choices.append(best_word)

        # Знаходимо чергову літеру повідомлення
        error = item['error']
        letter = get_letter(error, best_word)
        secret.append(letter)

    secret_message = ''.join(secret)

    return choices, secret_message

choices, secret_message = solve(model, vocab, lab.test_items)

lab.evaluate_accuracy(choices)
print("SECRET MESSAGE: ", secret_message)


<ipython-input-45-6cc23526443e>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.exp(-torch.tensor(total_log_prob) / len(tokens)).item()


Відповідь правильна ✅
accuracy = 0.66. Непогано! Спробуй розгадай приховане повідомлення
SECRET MESSAGE:   Зй правда кр ла     рунту н  тРебажзем   Не аєртор уде  ебо не    п ля т  б де  оля нем   Пари то  уду   х ари   цьом   апев   пра д  пт шина а як  е люд на а що ж л д на   ве н    млі  амар   літає   к и а ма    кр ла м єивони ті  р ла н  з пу уп р'  арз пр в и чес о   і дов р'  утк     з вірності у   ханні у когом зт ічногоіП риванзяіу  ог  ззощ рост о о Р б ти убкОго  з  е роСт  н  турботи   кого пз піСні або з  а ії аб ез поезії     З мріїолюдин   і и очне   т єо  к ил  ма  аакр л к а 


In [46]:
lab.answer("Ліна Костенко")

Відповідь правильна ✅
Правильно! 🚀 Заповни тепер цю форму, будь ласка: https://tally.so/r/wkl0zZ


Відправте посилання на цей colab або PDF з ним на пошту oleksii.o.syvokon@lpnu.ua. Дякую!
